In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

In [2]:
seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value) # gpu vars
torch.backends.cudnn.deterministic = True  #needed
torch.backends.cudnn.benchmark = False

## Load and prepare data

In [3]:
train_file = 'data/train.csv'
val_file = 'example/val.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.840B.300d'
config = {
    'max_seq': 40,
    'min_freq': 0,
    'batch_size': 51,
    'pretrained_file': pretrained_file
}


In [4]:
AttractiveData = AttractiveData(train_file, val_file, test_file, pretrained_file, config)

## Start to train

In [5]:
config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'CNN_LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 10
config['hidden_dim'] = 30
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 150
config['lr'] = {
    'encoder': 1e-5,
    'embedding': 6e-6,
    'linear': 1e-5
}
config['num_layers'] = 1
config['kernel_size'] = 3
config['dropout'] = 0.5
config['train_len'] = AttractiveData.train_len
config['val_len'] = AttractiveData.val_len
config['test_len'] = AttractiveData.test_len

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([12699, 300])


In [6]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, AttractiveData.valloader, pretrained_embeddings)

In [7]:
AttractiveTrainer.model, AttractiveTrainer.config['total_params'], AttractiveTrainer.config['total_learned_params']

(AttractiveNet(
   (embedding): AttractiveEmbedding(
     (token): TokenEmbedding(12699, 300, padding_idx=1)
   )
   (bigramcnn): Sequential(
     (0): Conv1d(300, 210, kernel_size=(2,), stride=(1,), padding=(1,))
     (1): BatchNorm1d(210, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (3): ReLU()
     (4): Conv1d(105, 55, kernel_size=(2,), stride=(1,), padding=(1,))
     (5): ReLU()
     (6): Dropout(p=0.5, inplace=False)
   )
   (trigramcnn): Sequential(
     (0): Conv1d(300, 210, kernel_size=(3,), stride=(1,), padding=(1,))
     (1): BatchNorm1d(210, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (3): ReLU()
     (4): Conv1d(105, 55, kernel_size=(3,), stride=(1,), padding=(1,))
     (5): ReLU()
     (6): Dropout(p=0.5, inplace=False)
   )
   (encoder_bigram): LSTM(55, 30,

In [8]:
AttractiveTrainer.train()

Epoch:   1%|          | 1/150 [00:00<01:01,  2.41it/s]
EP_0 | train loss: 0.9384422919329475 | val loss: 0.7165667028988109 |
Epoch:   1%|▏         | 2/150 [00:00<01:00,  2.44it/s]
EP_1 | train loss: 0.5792344205519733 | val loss: 0.5318450179754519 |
Epoch:   2%|▏         | 3/150 [00:01<00:59,  2.46it/s]
EP_2 | train loss: 0.5667228670681224 | val loss: 0.5237507446139467 |
Epoch:   3%|▎         | 4/150 [00:01<00:59,  2.47it/s]
EP_3 | train loss: 0.5785140944462196 | val loss: 0.525703467574774 |
Epoch:   3%|▎         | 5/150 [00:02<00:58,  2.49it/s]
EP_4 | train loss: 0.5525054510901956 | val loss: 0.5124210002375584 |
Epoch:   4%|▍         | 6/150 [00:02<00:57,  2.50it/s]
EP_5 | train loss: 0.5604159458010805 | val loss: 0.5169323565913182 |
Epoch:   5%|▍         | 7/150 [00:02<00:58,  2.44it/s]
EP_6 | train loss: 0.5489269144394818 | val loss: 0.5078892707824707 |
Epoch:   5%|▌         | 8/150 [00:03<01:00,  2.37it/s]
EP_7 | train loss: 0.5569075850879445 | val loss: 0.513976517845

## Below is testing

In [6]:
from attractivenet import AttractiveNet

PATH = './model/CNN_LSTM_20201110-212900/0.374438.34'

load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(12699, 300, padding_idx=1)
  )
  (bigramcnn): Sequential(
    (0): Conv1d(300, 210, kernel_size=(2,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(210, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
    (4): Conv1d(105, 55, kernel_size=(2,), stride=(1,), padding=(1,))
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
  )
  (trigramcnn): Sequential(
    (0): Conv1d(300, 210, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(210, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
    (4): Conv1d(105, 55, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
  )
  (encoder_bigram): LSTM(55, 30, batch_first=True, drop

In [7]:
def predict_attractive(sentence, category, phase):
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)
    tensor_sentence = tensor_sentence.unsqueeze(0)

    prediction = load_model(tensor_sentence, tensor_category, phase=phase)

    return prediction

In [8]:
predict_list = []
with torch.no_grad():
    for i, sentence in enumerate(AttractiveData.test_data):
        prediction = predict_attractive(sentence.Headline, sentence.Category, 'test')
        predict_list.append(prediction.item())

AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

## Below for statistics

In [9]:
import statistics
from sklearn.metrics import mean_squared_error

In [10]:
# # train mean = 3.15, test mean = 2.8
# train_list = []
# for i, sentence in enumerate(AttractiveData.train_data):
#     prediction = predict_attractive(sentence.Headline, sentence.Category, 'train')
#     train_list.append(prediction.item())
#     # train_list.append(prediction.item())
# # print(train_list)
# mean_squared_error(pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list(), train_list), statistics.mean(train_list), statistics.stdev(train_list)

In [11]:
# train_list[0:5], pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list()[0:5]

In [12]:
# a = AttractiveData.df_train['Label'].to_list()
# statistics.mean(a), statistics.stdev(a)

In [13]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.8865911372432627, 0.36633679291683463)

In [14]:
baseline_list = pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list()
mean_squared_error(baseline_list, predict_list), statistics.mean(baseline_list), statistics.stdev(baseline_list)

(0.02738899402714954, 2.8142020345259344, 0.36798823904910916)

In [17]:
mean_squared_error(pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list(), pd.read_csv('../309551062/predict/CNN_LSTM_20201109-125007_0.374958.75.csv').sort_values(['ID']).Label.to_list())

0.0